In [3]:
!pip install pyldavis
import pandas as pd
import numpy as np 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import re
import string
from gensim import corpora, models, similarities 

In [8]:
df = pd.read_csv('ar_reviews_100k.tsv',sep='\t')
df.sample(5)

,label,text
21037,Positive,الذكريات بكل ما تحمله من مرارة و سعادة إلا أن ...
47526,Mixed,أول كتاب اقرأه لمصطفى محمود، بالمجمل الكتاب جي...
83992,Negative,مكان يستحق السكن و الاستمتاع فية . الموقع جدا ...
81190,Negative,مخيب للأمل. . عدم وجود عوازل بين الغرف سماع ضج...
47806,Mixed,كتاب خفيف وبسيط يضم مجموعة من المقالات المتفرق...


In [10]:
df['text'] = df['text'].astype(str)
df['text'].head()

0    ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1    أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2    هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3    خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4    ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
Name: text, dtype: object

In [15]:
df['label'].value_counts()

Positive    33333
Mixed       33333
Negative    33333
Name: label, dtype: int64

In [17]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words('arabic')))
print(stop_words)

['بمن', 'صراحة', 'جيم', 'كأن', 'وا', 'إي', 'كأنما', 'لسنا', 'ز', 'إذا', 'عامة', 'واهاً', 'وهب', 'طرا', 'هذي', 'ستون', 'مكانكما', 'سبت', 'أربعمائة', 'إنَّ', 'ته', 'إياكم', 'بخٍ', 'أنت', 'هما', 'أخٌ', 'هم', 'ثلاثين', 'ياء', 'ما أفعله', 'هذين', 'قام', 'لن', 'كذا', 'سبحان', 'لا سيما', 'ثلاثمائة', 'فبراير', 'أنى', 'مما', 'حقا', 'طفق', 'ضاد', 'أف', 'إلا', 'ئ', 'خال', 'ذ', 'ذان', 'آهاً', 'بل', 'أنًّ', 'شيكل', 'كرب', 'في', 'ق', 'لها', 'مه', 'تينك', 'صباح', 'سمعا', 'خمسة', 'كانون', 'منه', 'دال', 'أن', 'مساء', 'خامس', 'جوان', 'خبَّر', 'أوشك', 'دولار', 'أيها', 'زود', 'أبٌ', 'كن', 'ذين', 'اثنين', 'أفٍّ', 'تعلَّم', 'أضحى', 'أيلول', 'أربعة', 'صبر', 'اربعين', 'لستم', 'أجمع', 'تحت', 'سبتمبر', 'سادس', 'إليكنّ', 'جلل', 'لنا', 'هنالك', 'مارس', 'درهم', 'أمس', 'ثان', 'فيها', 'ل', 'كيت', 'فو', 'حيَّ', 'ساء', 'ّأيّان', 'منذ', 'كأي', 'نعم', 'مئتان', 'ج', 'فإذا', 'لكي', 'سحقا', 'ثاني', 'حاشا', 'أى', 'لم', 'الألاء', 'ح', 'ليسوا', 'وهو', 'مايو', 'تسع', 'ذا', 'لكنَّ', 'لكن', 'ب', 'دون', 'يوان', 'اللتان', 'على', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
import re
import string
import sys
import argparse
from nltk.tokenize import word_tokenize

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [20]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

In [22]:
import nltk
nltk.download('punkt')
df['cleanedtext'] = df['text'].apply(clean_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
df.sample(10)

,label,text,cleanedtext
49320,Mixed,اديت للكتاب نجوم أول واحدة عشان الغلاف ألوانه ...,اديت للكتاب نجوم واحدة عشان الغلاف ألوانه عجبت...
23991,Positive,حكم تقودك لمعرفة الله جل علاه,حكم تقودك لمعرفة الله جل علاه
40980,Mixed,كتاب جميل لكن مليئ بالتفاصيل التى من الممكن ان...,كتاب جميل مليئ بالتفاصيل التى الممكن ان تجعل ا...
39968,Mixed,كل ما تحتاجه بجانبك . . المكان من حيث الموقع م...,تحتاجه بجانبك المكان الموقع ممتاز جدا يفتقر ال...
48948,Mixed,عندما تنتهى من الكتاب فى قعدة حينما تدوخ مع ال...,عندما تنتهى الكتاب فى قعدة حينما تدوخ الكتاب ل...
20199,Positive,رحلة عائلية . خدمة الموظفين. القوانين,رحلة عائلية خدمة الموظفين القوانين
85464,Negative,حسنا .. لم أكمل الكتاب بعد .. لكنني كونت فكرة ...,حسنا أكمل الكتاب لكنني كونت فكرة با ولية ، فقد...
37101,Mixed,من حيث الحدوته فهي مشوقة لكن صعب جدا أني اتعام...,الحدوته فهي مشوقة صعب جدا أني اتعامل الحدوته د...
39314,Mixed,مقبول. السعر معقول. حجزت عن طريق البوكنق سرير ...,مقبول السعر معقول حجزت طريق البوكنق سرير مزدوج...
10,Positive,انا بمووووووووووووووت ف حاجة اسمها أدهم صبري. ...,انا بمووووووووووووووت حاجة اسمها أدهم صبري الع...
